In [1]:
from playwright.async_api import async_playwright

pw = await async_playwright().start()
browser = await pw.chromium.launch(headless = False)
page = await browser.new_page()


# await page.screenshot(path="example.png")
# await browser.close()
# await pw.stop()

url1='https://m.facebook.com/wasitl'
url2='https://m.facebook.com/sothna.vicharueng'
url3='https://m.facebook.com/zuck'
url=url3

In [2]:
import time
# async with async_playwright() as pw:
browser = await pw.chromium.launch(headless = False)
page = await browser.new_page()
html = await page.goto(url)
await page.evaluate(
    '''
    var intervalID = setInterval(function () {
    var scrollingElement = (document.scrollingElement || document.body);
    scrollingElement.scrollTop = scrollingElement.scrollHeight;
    }, 200);
    ''')
prev_height = None
""" 
sometime facebook show page to login 
i make click to remove login page and continue scroll

make the mouse scroll up to make program restart
"""
try:
    item_close = await page.locator("[aria-label=\"Close\"]").click()
    if item_close is not None:
        item_close.click()
        page.mouse.wheel(-1000, 0)
except :
    pass
while True:
    curr_height = await page.evaluate('(window.innerHeight + window.scrollY)')
    if not prev_height:
        prev_height = curr_height
        # time.sleep(3)
    elif prev_height == curr_height:
        await page.evaluate('clearInterval(intervalID)')
        break
    else:
        prev_height = curr_height
        # time.sleep(10)

selector = await page.query_selector('div#root')
html = await selector.inner_html()
# selector = page.query_selector('div.rq0escxv.l9j0dhe7.du4w35lb')
# selector = page.query_selector('div.rq0escxv.l9j0dhe7.du4w35lb')
# I save html code in variable to parse it by beautifulSoup
# html = selector.inner_html()

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
username = soup.select_one('div#cover-name-root')
username.text

'Mark Zuckerberg'

In [4]:
import re
images = soup.select_one('div#profile_photos_unit').find_all('img')
img_urls = [img['src'] for img in images]
img_urls

['https://scontent.fbkk5-3.fna.fbcdn.net/v/t39.30808-6/305639385_10114682386840871_3268767058260494617_n.jpg?stp=cp0_dst-jpg_e15_q65_s206x206&_nc_cat=105&ccb=1-7&_nc_sid=5f2048&_nc_ohc=WlGUhzXPDZ8AX-4Rmy5&_nc_ht=scontent.fbkk5-3.fna&oh=00_AfDyyiBcEaWByQ9xryT7NHj-sP1Uym6YtRTms90iGy6UKQ&oe=65487561',
 'https://scontent.fbkk5-4.fna.fbcdn.net/v/t39.30808-6/283107467_10114484551689101_1906020902928950865_n.jpg?stp=cp0_dst-jpg_e15_p160x160_q65&_nc_cat=103&ccb=1-7&_nc_sid=5f2048&_nc_ohc=PTX7YpRi8VsAX_n2JtD&_nc_ht=scontent.fbkk5-4.fna&oh=00_AfCpac-qIcexEDfbS1LUu10LPcvTf73d7N4DazXXs6KLQA&oe=65480681',
 'https://scontent.fbkk5-8.fna.fbcdn.net/v/t39.30808-6/281913662_10114482955418041_2867919805640357486_n.jpg?stp=cp0_dst-jpg_e15_p100x100_q65&_nc_cat=106&ccb=1-7&_nc_sid=5f2048&_nc_ohc=K02K5v3GrUMAX_DkXJX&_nc_ht=scontent.fbkk5-8.fna&oh=00_AfCf-VQWIK5cEQjRld1LPI5CuMbU70l7CRxUnT335ZOAYg&oe=65495EFD',
 'https://scontent.fbkk5-5.fna.fbcdn.net/v/t1.6435-9/90633245_10111711606678131_6202964525352747008_

In [5]:
main = soup.select_one('div#main_column')
data={}
for i in main.select('div[data-sigil=profile-card]'):
    role=i.select_one('div[role=heading]').text
    # print(f'Role:{role}')
    
    ## exprerience
    exp=[]
    for j in i.select('div._5cds._2lcw'):
        d={}
        for k in j.select('div span'):    
            key='.'.join(k.get('class',''))
            if re.search('\w{1,}', key):
                # print(k)
                d[key]=k.text
        exp.append(d)
    if len(exp):
        data[role]=exp
    
    ## place lived
    live=[]
    for j in i.select('div._2swz._2lcw'):
        d={}
        for k in j.select('div header h4'):
            key='.'.join(k.get('class','h4'))
            if re.search('\w{1,}', key):
                # print(k)
                d[key]=k.text
        live.append(d)
    if len(live):
        data[role]=live
data

{'Work': [{'_52jd._52jb._52jh._3-8_': 'Chan Zuckerberg Initiative',
   '_52jc._52j9': '1 December 2015 - Present'},
  {'_52jd._52jb._52jh._3-8_': 'Meta',
   '_52jc._52ja': 'Founder and CEO',
   '_52jc._52j9': 'Palo Alto, California',
   '_52jc._52jb': 'Bringing the world closer together.'}],
 'Education': [{'_52jd._52jb._52jh._3-8_': 'Harvard University',
   '_52jc._52ja': 'Computer Science and Psychology',
   'concs.mfss.fcg': 'Computer Science and Psychology',
   '_52jc._52j9': '30 August 2002 - 30 April 2004'},
  {'_52jd._52jb._52jh._3-8_': 'Phillips Exeter Academy',
   '_52jc._52ja': 'Classics',
   'concs.mfss.fcg': 'Classics',
   '_52jc._52j9': 'School year 2002'},
  {'_52jd._52jb._52jh._3-8_': 'Ardsley High School',
   '_52jc._52ja': 'High School',
   '_52jc._52j9': 'September 1998 - June 2000'}],
 'Places lived': [{'h.4': 'Palo Alto, California',
   '_52jc._52ja._52jg': 'Current town/city'},
  {'h.4': 'Dobbs Ferry, New York', '_52jc._52ja._52jg': 'Home town'}],
 'About Mark': [{

In [ ]:
class Result:
    def __init__(self, value=None, error=None):
        self.value = value
        self.error = error

    @classmethod
    def of(cls, func, *args, **kwargs):
        try:
            return cls(value=func(*args, **kwargs))
        except Exception as e:
            return cls(error=str(e))

    @staticmethod
    def success(x):
        return Result(value=x)

    @staticmethod
    def failure(error):
        return Result(error=error)

    def is_error(self):
        return self.error is not None
    
    def bind(self, func):
        if self.is_error():
            return self
        return Result.of(func, self.value)

    def __repr__(self):
        if self.is_error():
            return f"Error({self.error})"
        return f"Success({self.value})"
    
    
def resultify(func):
    def wrapper(*args, **kwargs):
        return Result.of(func, *args, **kwargs)
    return wrapper